<a href="https://colab.research.google.com/github/harshit37/MS333_Ciao_Bella/blob/master/Video_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 1946 (delta 26), reused 35 (delta 16), pack-reused 1885
Receiving objects: 100% (1946/1946), 5.16 MiB | 3.11 MiB/s, done.
Resolving deltas: 100% (1256/1256), done.
Requirement already up-to-date: Cython in /usr/local/lib/python3.6/dist-packages (from -r yolov5/requirements.txt (line 2)) (0.29.21)
     |████████████████████████████████| 14.5MB 232kB/s 
     |████████████████████████████████| 43.7MB 65kB/s 
Requirement already up-to-date: torch>=1.6 in /usr/local/lib/python3.6/dist-packages (from -r yolov5/requirements.txt (line 5)) (1.6.0+cu101)
     |████████████████████████████████| 11.5MB 51.1MB/s 
     |████████████████████████████████| 2.2MB 46.1MB/s 
     |████████████████████████████████| 6.8MB 46.1MB/s 
     |████████████████████████████████| 276kB 54.9MB/s 
Requirement already up-to-date: torchvision>=0.6 in

In [ ]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/harshit37/SIH-2020

In [ ]:
import shutil
shutil.copy("/content/drive/My Drive/SIH/SIH Videos/Test_video.mp4","/content")

'/content/Test_video.mp4'

In [ ]:
import torch
from IPython.display import Image  # for displaying images   # for downloading models/datasets

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


In [ ]:
%cd yolov5

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5


In [ ]:
!python detect.py --source /content/Test_video.mp4 --weights /content/best_wm.pt --conf 0.6

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='/content/Test_video.mp4', update=False, view_img=False, weights=['/content/best_wm.pt'])
/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'models.yolo.Model' has changed. you can retrieve the original sourc

In [ ]:
# 3 files required -> coco.names,yolov3.weigths,yolov3.cfg and a sample image .. 
# link to files https://drive.google.com/drive/folders/1SJ084lQ7ACXnjnrG8Q8tkfvhm4oAma16?usp=sharing
# change the paths accordingly
# change the cv2_imshow function if running on laptop,on colab it will work fine



import numpy as np
import argparse
import time
from cv2 import cv2
import os



def extract_car(frame,writer):
  # path="data/vehicle-detector/voc.names"
   
  path="/content/drive/My Drive/yolo-object-detection/yolo-coco/coco.names"

  weightsPath = "/content/drive/My Drive/yolo-object-detection/yolo-coco/yolov3.weights"
  configPath = "/content/drive/My Drive/yolo-object-detection/yolo-coco/yolov3.cfg"
  
  #path = "src/alpr/alpr_data/vehicle-detector/yolo-coco/coco.names"
  labelsPath = path
  LABELS = open(labelsPath).read().strip().split("\n")

  np.random.seed(42)
  COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
    dtype="uint8")

  # weightsPath="data/vehicle-detector/yolo-voc.weights"
  # configPath ="data/vehicle-detector/yolo-voc.cfg"

 # weightsPath = "src/alpr/alpr_data/vehicle-detector/yolo-coco/yolov3.weights"
  #configPath = "src/alpr/alpr_data/vehicle-detector/yolo-coco/yolov3.cfg"

  print("Running vehicle-detector.py for database")
  net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)



#This is not the path of the image
#it is after cv2 has read the image


  image=frame																
  (H, W) = image.shape[:2]


  ln = net.getLayerNames()
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)


  boxes = []
  confidences = []
  classIDs = []
  keylist=['car', 'bus', 'truck', 'motorbike']        # Needs some changes
  dict_boundingbox = {key:[] for key in keylist}
  dict_confidence={key:[] for key in keylist}

  for output in layerOutputs:
    for detection in output:
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      if confidence > 0.5:
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))
        if LABELS[classID] in keylist:
          dict_boundingbox[LABELS[classID]].append([x, y, int(width), int(height)])
          dict_confidence[LABELS[classID]].append(float(confidence))
        

        confidences.append(float(confidence))
        boxes.append([x, y, int(width), int(height)])
        classIDs.append(classID)
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)

  if len(idxs) > 0:
    for i in idxs.flatten():
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
      text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
      cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, color, 2)
  img = image.copy()

  writer.write(img)
  print(dict_boundingbox)
  print(dict_confidence)
  cropped_images = []

  
  for key in keylist:
    if key=='motorbike':
      continue
    for bbox in dict_boundingbox[key]:
      x=bbox[0]
      y=bbox[1]
      w=bbox[2]
      h=bbox[3]
      # x,y 
      cropped_image=image[max(0,y):min(y+h,len(image)),max(0,x):min(x+w,len(image[1]))]
      cropped_images.append(cropped_image)





In [ ]:
import shutil
shutil.copy("/content/processed_video2.avi","/content/drive/My Drive")

'/content/drive/My Drive/processed_video2.avi'

In [ ]:
import numpy as np
from cv2 import cv2



path="/content/yolov5/inference/output/Test_video.mp4"
cap=cv2.VideoCapture(path)    
target = 200
counter = 1

writer=None
(W,H)=(None,None)

while True:
    print("counter" + str(counter))
    grabbed,frame=cap.read()

    if not grabbed:
        break

    if W is None or H is None:
        (H,W)=frame.shape[:2]

    if writer is None:
        fourcc=cv2.VideoWriter_fourcc(*"MJPG")
        writer=cv2.VideoWriter("/content/processed_video2.avi",fourcc,30,(W,H),True)

    extract_car(frame,writer)
    counter+=1
            



Streaming output truncated to the last 5000 lines.
{'car': [0.8209773302078247], 'bus': [], 'truck': [], 'motorbike': [0.8502675890922546, 0.9367479085922241, 0.6610164046287537]}
counter1005
Running vehicle-detector.py for database
{'car': [[1248, 70, 704, 601]], 'bus': [], 'truck': [], 'motorbike': [[1478, 409, 435, 592], [436, 545, 559, 481], [743, 113, 237, 315], [1, 439, 98, 323]]}
{'car': [0.6769120693206787], 'bus': [], 'truck': [], 'motorbike': [0.8024210333824158, 0.9083969593048096, 0.7497248649597168, 0.533119261264801]}
counter1006
Running vehicle-detector.py for database
{'car': [[1256, 67, 673, 607]], 'bus': [], 'truck': [], 'motorbike': [[173, 116, 420, 507], [426, 547, 559, 467], [1503, 545, 411, 465], [522, 7, 197, 276], [730, 104, 262, 333], [213, 242, 361, 399], [-6, 371, 133, 384]]}
{'car': [0.5776166319847107], 'bus': [], 'truck': [], 'motorbike': [0.6253854632377625, 0.9496272206306458, 0.5773703455924988, 0.7528972029685974, 0.5872834324836731, 0.7109967470169067

KeyboardInterrupt: ignored